In [1]:
import sys
import numpy as np

sys.path.insert(0, "..")

# Agents
from src.agents.robust_lin_exp3 import RobustLinExp3Agent

# Adversaries
from src.adversaries.adversary import Adversary
from src.adversaries.stochastic_linear_adversary import StochasticLinearAdversary

# Contexts
from src.contexts.context import Context
from src.contexts.gaussian_context import GaussianContext

# Environments
from src.environments.environment import Environment
from src.environments.adversarial_contextual_env import AdversarialContextualEnv

# Adversarial Contextual MAB bandits
from src.adversarial_contextual_mab import AdversarialContextualMAB

# Display
from src.display.visualization import Visualization as vis

In [7]:
# Init Vars
K = 10
h = 10000
epsilon = 0.0

# Init Context
num_ctx = 5
d = 3
ctx = GaussianContext(n_contexts=num_ctx, n_features=d, var=0.1)

# Init Agent
name = "RobustLinExp3"
lr = h**(-2/3)*(K*d)**(-1/3)*np.log(K)**(2/3)
exploration_rate = h**(-1/3)*(K*d*np.log(K))**(1/3)
sigma = ctx.estimate_sigma(n_samples=10000)
ag = RobustLinExp3Agent(name=name, K=K, d=d, sigma=sigma, eta=lr, gamma=exploration_rate, epsilon=epsilon) 

# Calculate upper bound
bound_per_timestep = np.array([ag.compute_regret_bound(t) for t in range(h)])

## StochasticLinearAdversary reset steps experiment

In [ ]:
# Init Adversary
noise_mean = epsilon
noise_std = 0.0
reset_steps = [-1, 1, 5, 10, 50]

# Get fixed theta
theta = np.random.randn(K, d)
theta /= np.maximum(np.linalg.norm(theta, axis=1, keepdims=True), 1.0) 

data_labels = []
cumulative_regrets = []
for reset_step in reset_steps:
    
    adv = StochasticLinearAdversary(
        num_actions=K, 
        context_dim=d, 
        noise_mean=noise_mean, 
        noise_std=noise_std, 
        reset_steps=reset_step, 
        theta=theta
    )
    
    # Init Environment
    adv_env = AdversarialContextualEnv(adversary=adv, context=ctx)
    
    # Play experiment
    num_sim = 100
    exp = AdversarialContextualMAB()
    _, _, _, _, cumulative_regret, _ = exp.play(agent=ag, environment=adv_env, num_sim=num_sim, horizon=h)

    # Append cumulative regret
    cumulative_regrets.append(cumulative_regret)
    data_labels.append(f"Reset steps = {reset_step}")

In [ ]:
vis.plot_mean_std(
    data=cumulative_regrets, 
    data_labels=data_labels,
    title=f"StochasticLinearAdversary with changing number of reset steps in {name}", 
    xlabel="n_steps", 
    ylabel="regrets", 
    baseline=bound_per_timestep,
    baseline_title=f"{name} regret upper bound"
)